<a href="https://colab.research.google.com/github/Muhammadsulton1/MIPT_Data_analys/blob/main/%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%D1%81_%D0%BB%D0%BE%D0%B3%D0%B0%D0%BC%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


##Работа с логами
Логи - это файлы с записью различных событий в хронологическом порядке. Обычно, все действия пользователя в различных сервисах подробно логируются. Именно по логам в дальнейшем аналитики считают различные метрики и строят модели.

В этой тетрадке мы проанализируем логи покупок онлайн-магазина, продающего атрибутику компании Google.
данные:https://www.kaggle.com/c/ga-customer-revenue-prediction/data?select=train.csv

Цели:

Построить несколько пользовательских метрик: посещаемость, средняя выручка, возвращаемость
Проанализровать как ведут себя пользовательские метрики в разбивке по разным источникам (браузеры, устройства и тп)
Сделать исходя из этого анализа несколько мудрых маркетинговых выводов




In [ ]:
import numpy as np         # библиотека для матриц и математики
import pandas as pd        # библиотека для работы с табличками
from scipy import stats    # модуль для работы со статистикой

import matplotlib.pyplot as plt
import seaborn as sns

# стиль графиков
# plt.style.use('ggplot')
plt.style.use('fivethirtyeight')
%matplotlib inline


1. Данные
Познакомимся с логом, с которым мы будем работать. Приведём его в удобный для строительства метрик вид.

In [ ]:
df = pd.read_csv('google_log.csv', sep='\t')

df['totals.transactionRevenue'] = df['totals.transactionRevenue']/10**6

print(df.shape)
df.head()

Описание колонок:

date дата посещения сайта (рассматривается период с 20160801 по 20170801

fullVisitorId уникальный id пользователя

sessionId уникальный id одной пользовательской сессии

channelGrouping откуда произошёл переход

visitStartTime timestamp начала визита

device.browser браузер визита

device.operatingSystem операционная система устройства

device.isMobile является ли устройство мобильным

device.deviceCategory тип устройства (айпад, компьютер, мобильный телефон)

geoNetwork.subContinent часть света пользователя

geoNetwork.country страна пользователя

geoNetwork.region регион пользователя

geoNetwork.city город пользователя

totals.hits похоже что это действия на сайте, но это неточно

totals.pageviews просмотры страниц

totals.transactionRevenue выручка с покупки

trafficSource.source источник трафика

trafficSource.medium более высокоуровневый источник трафика

trafficSource.keyword ключевые слова из поиска

trafficSource.adwordsClickInfo.adNetworkType несколько переменных с дополнительной информацией из adwords

trafficSource.adwordsClickInfo.page

trafficSource.adwordsClickInfo.slot

trafficSource.adwordsClickInfo.isVideoAd

trafficSource.adContent

Посмотрим на процент пропусков.

In [ ]:
100*df.isnull().sum()/df.shape[0]


Визуализируем пропуски в различных колонках. Сделаем случайную подвыборку в $5000$ строк и раскрасим пропуски в жёлтый.

In [ ]:

fig, ax = plt.subplots(figsize=(15,8))

sns_heatmap = sns.heatmap(df.sample(5000).isnull(),
                          yticklabels=False,
                          cbar=False,
                          cmap='viridis')

Посмотрим на описательные статистики.

In [ ]:
df.describe()

In [ ]:
df.describe(include='object')

Заведём несколько переменных со временем.

In [ ]:
df["date"].dtype

In [ ]:
df['date'][:10]

In [ ]:
df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")
df["date"][:10]

Модуль dt включает в себя довольно много встроенных в pandas методов для работы со временем.

In [ ]:
df['month'] = df['date'].apply(lambda w: w.strftime('%Y-%m'))
df['month'][:10]

In [ ]:
df["visitDay"] = df['date'].dt.day
df["visitDay"][:10]

In [ ]:
df["visitDay"] = df['date'].dt.day             # день визита
df["visitMonth"] = df['date'].dt.month         # месяц визита

df["visitWeekday"] = df['date'].dt.weekday     # выходные
df["visitWeeknum"] = df['date'].dt.weekofyear  # порядковый номер недели в году

# начало и конец месяца
df["is_month_start"] = df['date'].dt.is_month_start
df["is_month_end"] = df['date'].dt.is_month_end

In [ ]:
df['visitStartTime'][:10]

In [ ]:
from datetime import datetime
datetime.fromtimestamp(1472812272).minute

In [ ]:
# час визита
df['visitHour'] = (df['visitStartTime'].apply(
    lambda x: datetime.fromtimestamp(x).hour))

# время визита с точностью до секунды
df['ts'] = df['visitStartTime'].apply(lambda x:
                                      pd.datetime.fromtimestamp(x))


#2. Возвращаемость и посещаемость
Проанализируем что происходит с посещаемостью сайта и возвращаемостью пользователей.

In [ ]:
df.head()


#2.1 Сколько людей пользуются магазином в день/месяц?¶
Посчитаем уникальное число пользователей для каждого дня.

In [ ]:
# для большей читаемости кода
df_day = (
    df.groupby('date')
    .agg({'fullVisitorId': 'nunique'})
    .sort_values('date')
)

df_day.head(

In [ ]:
print(f"Среднее число посетителей в день: {df_day.fullVisitorId.mean()}")
df_day.plot(figsize=(12,6));

In [ ]:
df_month = (
    df.groupby('month')
    .agg({'fullVisitorId': 'nunique'})
    .sort_values('month')
)

print(f"Среднее число посетителей в месяц: {df_month.fullVisitorId.mean()}")
df_month.plot(figsize=(12,6));


Выводы: Основаня волна посещений магазина наблюдается осенью, летом идёт спад. На дневном графике чётко видна недельная сезонность. В выходные наблюдается спад пользовательской активности.

#2.2 Сколько пользовательских сессий в день?
Для этого нам необходимо сгруппировать по дням посещения и посчитать кол-во пользователей, но не уникальных.

In [ ]:
df_session = (
    df.groupby('date')
    .agg({'fullVisitorId': 'count'})
    .sort_values('date')
)

print(f"Среднее число сессий в день: {df_session.fullVisitorId.mean()}")
df_session.plot(figsize=(12,6));


Видим, что график не сильно отличается от динамики уникальных пользователей. Это позволяет предположить, то что возвращаются люди очень редко. Проверим это.

#2.3 Как часто люди возвращаются?¶
Чтобы ответить на этот вопрос воспользуемся когортным анализом, за когорту будем брать посетителей зарегистровавшихся в одном месяце.

In [ ]:
first_month_session = (
    df.groupby('fullVisitorId')
    .agg({
        'month':'min',
        'date': 'min',
        'ts': 'min'
    })
)

first_month_session.columns = ['first_invate_month', 'first_invate_day', 'first_invate_ts']
first_month_session.head()

In [ ]:
df = df.join(first_month_session, on='fullVisitorId')

kogort_month = df[['fullVisitorId', 'month', 'first_invate_month']]
kogort_month.head()


Выделили группы уникальных пользователей по месяцам, построим таблицу, которая покажет возврат пользователей на ресурс.

In [ ]:
retention = kogort_month.pivot_table(
    index='first_invate_month',
    columns='month',
    values = 'fullVisitorId',
    aggfunc='nunique'
)

retention

In [ ]:
plt.figure(figsize=(10, 8))

sns.heatmap(
    retention,
    annot=True,
    fmt='.0f',
    linewidths=1,
    linecolor='black',
    cmap="YlGnBu"
);

Перейдём к процентам.

In [ ]:
first_date = retention.values.diagonal()

for s,row in zip(first_date, retention):
    retention.loc[row] = 100*(retention.loc[row]/s)

plt.figure(figsize=(10, 8))
sns.heatmap(
    retention,
    annot=True,
    fmt='.2f',
    linewidths=1,
    linecolor='black',
    cmap="YlGnBu"
);

Возвраты пользователей крайне малы. Где-то 3% возвращается на второй месяц, а дальше возврат обваливатеся почти до нуля.

Какая доля визитов относится к месяцу отличному от первого визита?

In [ ]:
(kogort_month.month != kogort_month.first_invate_month).mean()

А доля дней?

In [ ]:
(df.date != df.first_invate_day).mean()

Видим, что повторные визиты бывают довольно редко.

#3. Покупки
Проанализируем покупки, которые делаются на сайте.

#3.1 Когда люди покупают?

In [ ]:
df['totals.transactionRevenue'].isnull().mean()

In [ ]:
# Процент покупок
100*(1 - df['totals.transactionRevenue'].isnull().mean())

Около 1% пришедших сделали покупку. Такова суровая реальность.

In [ ]:
df_buy = df.dropna(subset=['totals.transactionRevenue'])
df_buy.shape


Посморим на распределение выручки.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16,6))

df_buy['totals.transactionRevenue'].hist(bins=50, log=True, ax=axes[0])
sns.distplot(
    np.log(df_buy['totals.transactionRevenue'] + 1),
    ax=axes[1],
    bins=100,
    kde=True)

axes[0].set_title("Распределение выручки")
axes[0].set_xlabel("Выручка")

axes[1].set_title("Распределение логарифма выручки")
axes[1].set_xlabel("Логарифм выручки")

Посмотрим на дату первой покупки. Как часто она отличается от даты первого визита.

In [ ]:
first_day_buy = (
    df_buy.groupby('fullVisitorId')
    .agg({
        'month':'min',
        'date': 'min',
        'ts': 'min'
    })
)

In [ ]:
first_day_buy.head()

In [ ]:
first_day_buy.columns = ['first_buy_month', 'first_buy_day', 'first_buy_ts']
first_day_buy.head()

In [ ]:
df_buy = df_buy.join(first_day_buy, on='fullVisitorId')
df_buy.head()

In [ ]:
(df_buy.first_invate_day != df_buy.first_buy_day).mean()

Сколько дней проходит от первого захода на сайт до первой покупки?

In [ ]:
(df_buy['first_buy_ts'] - df_buy['first_invate_ts']).head()

In [ ]:
df_buy['deltatime_buy'] = (df_buy['first_buy_ts'] - df_buy['first_invate_ts'])/np.timedelta64(1,'D')

df_buy['deltatime_buy'].hist(bins=30)
plt.xlabel('Время между первой покупкой и входом на сайт');

In [ ]:
df_buy['deltatime_buy'].describe()

Выбросы сильно искажают среднее.

А часов?

In [ ]:
df_buy['deltatime_buy'] = (df_buy['first_buy_ts'] - df_buy['first_invate_ts'])/np.timedelta64(1,'h')
df_buy['deltatime_buy'].hist(bins=30)
plt.xlabel('Время между первой покупкой и входом на сайт');

In [ ]:
df_buy['deltatime_buy'].describe()

In [ ]:
df_buy['deltatime_buy'].quantile(0.6)

Видим, что свою первую покупку пользователи совершают чаще всего в первый же визит. В $40\%$ случаев между визитом, когда была совершена покупка и первым визитом проходит больше 2 часов. Также мы видим, что хвост у распределения оказывается довольно большим.

Посмотрим на динамику покупок.

In [ ]:
df_buy.groupby('date')['fullVisitorId'].count().plot(figsize=(12,6));

График покупок не отличается существенно от графика визитов в плане своей динамики. Посмотрим на то, как покупки распределены по дням недели и часам.

In [ ]:
df_buy.pivot_table(
    index = 'visitHour',
    columns = 'visitWeekday',
    values = 'totals.transactionRevenue',
    aggfunc='sum'
).style.background_gradient()


C 17 до 23 часов видим самое большое число покупок.

#3.2 Сколько раз покупают за период?
Посчитаем статистику о покупках в разбиении по когортам. Когорта - месяц первой покупки.

In [ ]:
sales = df_buy.pivot_table(
    index='first_buy_month',
    columns='month',
    values = 'fullVisitorId',
    aggfunc='nunique'
)

plt.figure(figsize=(10, 8))
sns.heatmap(
    sales,
    annot=True,
    fmt='.0f',
    linewidths=1,
    linecolor='black',
    cmap="YlGnBu"
);


Снова видим, что интерес к магазину падает уже на второй месяц.

#3.3 Средний доход
Посчитаем его на дневной основе.

In [ ]:

g = (
    df_buy.groupby('date')
    .agg({'totals.transactionRevenue':'mean'})
    .plot(figsize=(12,6))
)

Посчитаем сколько денег приносят в среднем отдельные когорты. Для этого посчитаем суммарную выручку с каждой и поделим на число пользователей в ней.

In [ ]:
sales_sum = df_buy.pivot_table(
    index='first_buy_month',
    columns='month',
    values = 'totals.transactionRevenue',
    aggfunc='sum'
)

plt.figure(figsize=(10, 8))
sns.heatmap(
    sales_sum/sales,
    annot=True,
    fmt='.0f',
    linewidths=1,
    linecolor='black',
    cmap="YlGnBu"
);

Есть несколько покупок на очень крупную сумму в февральской когорте.

#4. Анализ выручки в разбивке по источникам
Проанализируем выручку в разбивке по её источникам.

In [ ]:
df_buy['totals.transactionRevenue'] = df_buy['totals.transactionRevenue'].apply(lambda w: np.log(w + 1))

Посмотрим какие браузеры самые популярные.

In [ ]:
df_buy['device.browser'].value_counts()[:10]

In [ ]:
plt.figure(figsize=(12,6))

df_buy['device.browser'].value_counts().plot(kind='bar')

plt.title("TOP 10 самых используемых браузеров", fontsize=20)
plt.xlabel("Браузер", fontsize=16)
plt.ylabel("Число визитов", fontsize=16);


Выручка с разбивкой по браузерам

In [ ]:
plt.figure(figsize=(12,6))

data = (df_buy[
    df_buy['device.browser'].isin(
        df_buy['device.browser'].value_counts()[:10].index.values
    )]
)

g = sns.boxplot(x='device.browser',
              y='totals.transactionRevenue',
              data=data);

g.set_title('Выручка в разбивке по браузерам', fontsize=20)
g.set_xlabel('Браузер', fontsize=18)
g.set_ylabel('Распределение выручки', fontsize=18);

Посмотрим на операционные системы.

In [ ]:
plt.figure(figsize=(12,6))

df_buy['device.operatingSystem'].value_counts().plot(kind='bar')

plt.title("Операционная система", fontsize=20)
plt.xlabel("Операционная система", fontsize=16)
plt.ylabel("Число визитов", fontsize=16);

Прибыль по разным операционным системам

In [ ]:
data = (
    df_buy[
        df_buy['device.operatingSystem']
       .isin(df_buy['device.operatingSystem'].value_counts()[:6].index.values)
          ]
)


# Такой же приём можно использовать и для графиков
g = (
    sns.FacetGrid(data, hue='device.operatingSystem', aspect=2)
    .map(sns.kdeplot, 'totals.transactionRevenue', shade=True)
    .add_legend()
);

Выручка в разбивке по разным типам устройств.

In [ ]:
sns.countplot(df_buy["device.deviceCategory"], palette="hls");

In [ ]:
plt.figure(figsize=(12,6))

g = sns.boxplot(x='device.deviceCategory',
              y='totals.transactionRevenue',
              data=df_buy);

g.set_title('Выручка в разбивке по типам устройств', fontsize=20)
g.set_xlabel('Устройство', fontsize=18)
g.set_ylabel('Распределение выручки', fontsize=18);

In [ ]:
g = (
    sns.FacetGrid(df_buy, hue='device.deviceCategory', aspect=2)
    .map(sns.kdeplot, 'totals.transactionRevenue', shade=True)
    .add_legend()
);

Из каких городов чаще всего делаются покупки.

In [ ]:
# !pip install squarify

In [ ]:
df_buy["geoNetwork.city"].value_counts()

In [ ]:
import squarify

df_cur = df_buy[df_buy["geoNetwork.city"] != 'not available in demo dataset']

In [ ]:
city_tree = df_cur["geoNetwork.city"].value_counts()
city_tree = round((city_tree[:30] / len(df_cur['geoNetwork.city']) * 100),2)

plt.figure(figsize=(12,12))

g = squarify.plot(
    sizes=city_tree.values,
    label=city_tree.index,
    value=city_tree.values,
    alpha=.3
)

g.set_title("Топ-30 городов по выручке - % от общей суммы", fontsize=20)
g.set_axis_off();